In [133]:
import numpy as np
# from sklearn.cluster import KMeans
import networkx as nx
import pandas as pd
from sklearn.datasets import load_digits
from minisom import MiniSom

### Source Paper:  
"Enriched topological learning for cluster detection and visualization"  
Guénaël Cabanes, Younès Bennani, Dominique Fresneau  
10.1016/j.neunet.2012.02.019 

In [134]:
mnist_X = load_digits().images.reshape(1797, -1)
mnist_y = load_digits().target

In [135]:
def get_prototypes(data, SOM_dim=None):
    """Define SOM and train on data.

    Input:
        - Data, SOM dimensions

    Output: 
        - Trained SOM Object
    """
    SOM = MiniSom(SOM_dim, SOM_dim, data.shape[1])
    SOM.pca_weights_init(data)
    SOM.train(data=data, num_iteration=100000)
    return SOM

In [156]:
data = mnist_X
SOM_dim = 15

if SOM_dim is None:
    sample_size = len(data)
    #  n_neurons = 5 * sqrt(sample_size)
    #  dim = sqrt(n_neurons)
    SOM_dim = int((5*(sample_size**(1/2)))**(1/2))

SOM = get_prototypes(data, SOM_dim=SOM_dim)
Dist = SOM._distance_from_weights(data).T

In [157]:
def enrich_prototypes(Dist, sigma=None):
    """
    Input: 
        - Distance Matrix Dist(w, x) between M prototypes w und the N data x

    Output: 
        - The density D_i and local variability s_i associated to each prototype w_i,  
        - The neighborhood values v_{i,j} associated with each pair of prototypes w_i and w_j
    """
    d = estimate_density(Dist=Dist, sigma=sigma, SOM=SOM)
    s = estimate_local_variability(Dist=Dist, SOM=SOM)
    v = estimate_neighborhood_values(Dist)

    prototypes = pd.DataFrame([s, d], index=["s", "d"]).T
    v = pd.DataFrame(v)

    return v, prototypes

$D_i = 1/N * \sum_{k=1}^N \frac{e^{-\frac{{Dist(w_i, x_k)^2}}{2\sigma^2}}}{\sigma \sqrt{2\pi}}$  
$D_i =  \frac{1/N }{\sigma \sqrt{2\pi}} * \sum_{k=1}^N e^{-\frac{{Dist(w_i, x_k)^2}}{2\sigma^2}}$

In [158]:
def estimate_density(Dist, SOM, sigma=None):
    """Estimate local density for each prototype from its assigned samples."""
    #  Default value: Average distance between prototype and nearest neighbor.
    if sigma is None:
        sigma = Dist.min(axis=1).mean()

    indices = SOM.win_map(data, return_indices=True)
    D =  np.zeros(shape=(SOM_dim, SOM_dim))

    for prototype_index, samples_indices in indices.items():
        index_flat = np.ravel_multi_index(prototype_index, (SOM_dim, SOM_dim))
        neighbors = Dist[index_flat, samples_indices]
        # print(neighbors)
        neighbors = neighbors**2
        neighbors = np.e ** -(neighbors/(2*sigma**2))
        neighbors = neighbors / sigma*np.sqrt(2*np.pi)
        neighbors = np.mean(neighbors)

        D[prototype_index] = np.mean(neighbors)

    return D.flatten()

In [159]:
def estimate_neighborhood_values(Dist):
    """For each data x, find the two clostest prototypes u*(x) and u**(x) (Best Matching Units, BMUs).

    Compute the number v_{i,j} of data having i and j as first two BMUs
    """
    BMUs = np.argsort(Dist, axis=0)[:2,:]
    v = np.zeros(shape=(SOM_dim**2, SOM_dim**2))
    u, counts = np.unique(BMUs, axis=1, return_counts=True)
    u = u.T
    for index, combination in enumerate(u):
        v[combination[0], combination[1]] = counts[index]

    return v

In [160]:
def estimate_local_variability(Dist, SOM):
    """For each prototype w, variability s is the mean distance 
    between w and the L data x_w represented by w.

    s_i = 1/L \sum(j=1..L) Dist(w_i, x_wi)
    """
    indices = SOM.win_map(data, return_indices=True)
    s = np.zeros(shape=(SOM_dim, SOM_dim))

    for prototype_index, samples_indices in indices.items():
        index_flat = np.ravel_multi_index(prototype_index, (SOM_dim, SOM_dim))
        s[prototype_index] = Dist[index_flat, samples_indices].mean()

    return s.flatten()

In [161]:
v, prototypes = enrich_prototypes(Dist)

In [162]:
def extract_connected_units(v, threshold):
    """Find all L groups of linked prototypes i, j where v_{i,j} > treshold.
    Input:
        - v: Matrix of connected components (v_{i,j} have common samples).
    """
    indices = np.where(v > threshold)
    groups = set()
    for i in zip(indices[0], indices[1]):
        groups.add((i))
    groups = pd.DataFrame(groups)

    return groups

In [163]:
neighborhood_list = extract_connected_units(v, threshold=0)

In [164]:
def create_graph(groups, prototypes):
    """Create Graph with edges between prototypes. Edges are directed from
    high density nodes to low density nodes with a positive gradient.

    Input:
        - The list of connected prototypes,
        - Data for each prototype
    Output:
        - Graph object with protoypes and gradients between prototypes
    """
    for i in range(len(groups)):
        #  Negative gradient for Label Propagation
        groups.loc[i, "gradient"] = -(prototypes.d[groups.loc[i, 0]] - prototypes.d[groups.loc[i, 1]])

    positive_edges = groups[groups.gradient > 0]
    #  reverse edge direction for later algorithms
    G = nx.from_pandas_edgelist(positive_edges, source=1, target=0, edge_attr="gradient", create_using=nx.DiGraph)

    nx.set_node_attributes(G, prototypes.d, "density")
    nx.set_node_attributes(G, prototypes.s, "variability")

    return G

In [165]:
G = create_graph(neighborhood_list, prototypes)

In [166]:
def initial_clustering(G):
    """Label each prototype by the maximum with the highest gradient.

    Input: 
        - Graph

    Output:
        - None
    """
    #  Create initial labels
    for node in G:
        G.nodes[node]["label"] = node

    #  Number of needed iterations
    longest_path = nx.algorithms.dag.dag_longest_path_length(G)

    for i in range(longest_path):
        #  Iterate over (node, neighbor) pairs
        for node, edges in G.pred.items():

            #  get largest gradient neighbor
            largest_gradient = 0
            largest_gradient_neighbor = node
            for edge in edges.items():
                current_neighbor = edge[0]
                current_gradient = edge[1]["gradient"]

                if current_gradient > largest_gradient:
                    largest_gradient = current_gradient
                    largest_gradient_neighbor = current_neighbor

            G.nodes[node]["label"] = G.nodes[largest_gradient_neighbor]["label"]

initial_clustering(G)

In [167]:
labels = pd.DataFrame(G.nodes(data="label"))
labels = set(labels.loc[:,1].unique())

In [168]:
def find_maxima(G):
    """Find all Nodes which only have positive (negative internally) outgoing gradients.
    ToDo: Check signs

    Input: 
        - Graph

    Output:
        - List of maxima nodes
    """
    maxima = []
    for node in G:
        if G.in_degree(node) == 0:
            maxima.append(node)

    return maxima

In [169]:
maxima = find_maxima(G)

In [170]:
def check_gradients(G):
    """Check if gradients in the edge weights are the same as differences between gradients stored in nodes.

    Input:
        - Graph
    """
    for node, nbrs in G.adj.items():
        for nbr, edge_atrr in nbrs.items():
            gradient =  G.nodes[nbr]["density"] - G.nodes[node]["density"]
            if edge_atrr["gradient"] != -gradient:
                print("Wrong gradient found")

check_gradients(G)

In [171]:
print("labels", list(set([val[1] for val in G.nodes.data("label")])))
n_labels = len(list(set([val[1] for val in G.nodes.data("label")])))
print(f"Number of labels: {n_labels}")

labels [3, 7, 138, 10, 142, 16, 144, 149, 150, 154, 28, 30, 167, 39, 170, 173, 175, 177, 183, 58, 189, 62, 191, 64, 65, 194, 67, 201, 202, 74, 207, 81, 213, 85, 91, 222, 224, 98, 108, 114, 116, 121, 125]
Number of labels: 43


In [172]:
def merge_micro_clusters(G, label_i, label_j, density_i, density_j):
    #  Get larger density label
    if density_i > density_j:
        new_label = label_i
        old_label = label_j
    else:
        new_label = label_j
        old_label = label_i

    #  Overwrite lower density label
    for node, label in G.nodes.data("label"):
        if label == old_label:
            G.nodes[node]["label"] = new_label


In [173]:
def final_clustering(G):
    """Merge clusters according to density threshold of the clusters.
    
    Input:
        - Graph
    """
    for e in G.edges:
        node_i = e[0]
        node_j = e[1]

        label_i = G.nodes[node_i]["label"]
        label_j = G.nodes[node_j]["label"]

        density_i = G.nodes[node_i]["density"]
        density_j = G.nodes[node_j]["density"]

        density_max_i = G.nodes[label_i]["density"]
        density_max_j = G.nodes[label_j]["density"]

        threshold = (1/density_max_i + 1/density_max_j)**-1

        if (density_i > threshold and 
            density_j > threshold and
            label_i != label_j):
                merge_micro_clusters(G, label_i, label_j, density_i, density_j)

final_clustering(G)

In [174]:
print("labels", list(set([val[1] for val in G.nodes.data("label")])))
n_labels = len(list(set([val[1] for val in G.nodes.data("label")])))
print(f"Number of labels: {n_labels}")

labels [3, 7, 74, 175, 62, 191]
Number of labels: 6


In [175]:
pd.DataFrame([val for val in G.nodes.data("label")])

,0,1
0,213,62
1,198,62
2,30,3
3,15,3
4,115,62
...,...,...
203,108,62
204,52,62
205,56,62
206,5,3
